In [1]:
import sys
import os
import cv2
from pathlib import Path
from PIL import Image, ImageSequence
import numpy as np
from ultralytics import YOLO

In [2]:
# Find pyproject.toml in parent directories and add its directory to sys.path
path_curr = Path.cwd()
for parent in [path_curr] + list(path_curr.parents):
    path_pyproject = parent / "pyproject.toml"
    if path_pyproject.exists():
        sys.path.insert(0, str(parent))
        break

In [3]:
# Create output directory and save result images
dir_output = path_curr.parent / "output"
dir_output.mkdir(exist_ok=True)

In [4]:
# Load a pre-trained YOLO model (e.g., YOLOv8s for small, yolov8n for nano)
model = YOLO("yolo12l.pt") 

In [5]:
path_gif: Path = Path("../images/rabbits.gif")

In [6]:
# Open GIF and iterate frames
im = Image.open(path_gif)


In [7]:
frames_out = []
durations = []

In [8]:
def shrink_image(arr_input, dim_max=320):
    h, w = arr_input.shape[:2]
    if max(h, w) > dim_max:
        scale = dim_max / max(h, w)
        w_new, h_new = int(w * scale), int(h * scale)
        arr_output = cv2.resize(arr_input, (w_new, h_new), interpolation=cv2.INTER_AREA)
    else:
        arr_output = arr_input
    return arr_output

In [9]:
for frame in ImageSequence.Iterator(im):
    durations.append(frame.info.get("duration", 100))  # ms per frame (default 100)
    rgb = frame.convert("RGB")
    arr_shrinked = np.array(rgb)

    # shrink image if larger than max dimension
    arr_shrinked = shrink_image(arr_shrinked, dim_max=320)

    # Run YOLO prediction on the frame (array in RGB)
    results = model(arr_shrinked)
    # Get annotated image (numpy array)
    annotated = results[0].plot()

    # Convert annotated array back to PIL Image and collect
    frames_out.append(Image.fromarray(annotated))



0: 640x384 1 bowl, 56.9ms
Speed: 1.5ms preprocess, 56.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 1 bowl, 36.9ms
Speed: 1.0ms preprocess, 36.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 1 bowl, 36.9ms
Speed: 1.1ms preprocess, 36.9ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 sheeps, 1 bowl, 36.6ms
Speed: 1.1ms preprocess, 36.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 31.5ms
Speed: 1.2ms preprocess, 31.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bowl, 31.3ms
Speed: 1.1ms preprocess, 31.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 1 bowl, 30.9ms
Speed: 1.0ms preprocess, 30.9ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bowl, 30.7ms
Speed: 1.0ms preprocess, 30.7ms inference, 1.5ms postprocess per im

In [10]:
# Save annotated frames back to a GIF
if frames_out:
    path_save = dir_output / f"{path_gif.stem}_predicted.gif"
    frames_out[0].save(
        path_save,
        save_all=True,
        append_images=frames_out[1:],
        duration=durations,
        loop=0,
        optimize=False,
    )
    print("Saved predicted GIF to:", path_save)
else:
    print("No frames extracted from:", path_gif)

Saved predicted GIF to: /home/masayuki/Documents/Projects/Python/YOLO_camera/output/rabbits_predicted.gif
